In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, BatchNormalization, Bidirectional, LayerNormalization, Add, Attention
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load and preprocess your dataset
data = pd.read_csv('/DSL-StrongPasswordData.csv')

label_encoder = LabelEncoder()
data['subject'] = label_encoder.fit_transform(data['subject'])

X = data.drop(columns=['subject', 'sessionIndex', 'rep'])
y = data['subject']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.20, random_state=42)


input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))


x = Bidirectional(GRU(128, activation='relu', return_sequences=True))(input_layer)
x = Dropout(0.5)(x)
residual_1 = LayerNormalization()(x)


x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(residual_1)
x = Dropout(0.4)(x)


x_res = Dense(128)(residual_1)  
x_res = tf.keras.layers.Reshape((x_res.shape[1], 128))(x_res)  
x = Add()([x, x_res]) 
residual_2 = LayerNormalization()(x)



x = tf.keras.layers.Flatten()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

output_layer = Dense(51, activation='softmax')(x)


model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=1e-6)


history = model.fit(
    X_train, y_train,
    epochs=250,
    batch_size=128,

    callbacks=[early_stopping, lr_scheduler],
    verbose=1
)


test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Test accuracy:", test_accuracy)


Epoch 1/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 15s 36ms/step - accuracy: 0.2376 - loss: 3.1709 - learning_rate: 0.0010
Epoch 2/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6409 - loss: 1.3321 - learning_rate: 0.0010
Epoch 3/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7182 - loss: 1.0123 - learning_rate: 0.0010
Epoch 4/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7657 - loss: 0.8450 - learning_rate: 0.0010
Epoch 5/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7811 - loss: 0.7616 - learning_rate: 0.0010
Epoch 6/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8014 - loss: 0.6947 - learning_rate: 0.0010
Epoch 7/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8121 - loss: 0.6486 - learning_rate: 0.0010
Epoch 8/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8250 - loss: 0.5917 - learning_rate: 0.0010
Epoch 9/250
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8291 - loss: 0.5799 - learning_rate: 0.0010